In [1]:
import os
import torch

import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib
%matplotlib


Using matplotlib backend: MacOSX


### Here is the global MVaR PF

In [2]:
fig, ax = plt.subplots(1, figsize=(5, 4))
all_mvars = torch.load(os.path.join("max_hvs", "disc_brake_all_mvars.pt"))
ax.scatter(all_mvars[:, 0], all_mvars[:, 1])
ax.set_title("Disc Brake MVaR PF")
fig.tight_layout()

### Let's draw 100 points from the domain and see how things look for those.

In [3]:
import json
from approximate_max_hv_run import construct_mvar_hv, exp_dir

config_path = os.path.join(exp_dir, "disc_brake", "config.json")
with open(config_path, "r") as f:
    config = json.load(f)
config.pop("device")
mvar_hv = construct_mvar_hv(**config, use_cpu=True)

In [4]:
from botorch.utils.sampling import draw_sobol_samples

standard_bounds = torch.ones(2,4)
standard_bounds[0] = 0

X_test = draw_sobol_samples(bounds=standard_bounds, n=100, q=1).squeeze(-2)


In [5]:
# Get the corresponding MVaR set.
perturbed_X = mvar_hv.perturbation(X_test)
perturbed_Y = mvar_hv.eval_problem(perturbed_X)

infeas = (perturbed_Y[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
perturbed_Y = perturbed_Y[..., : -mvar_hv.num_constraints]
perturbed_Y[infeas] = mvar_hv.hv.ref_point

mvar = (
    mvar_hv.mvar(
        perturbed_Y.cpu(),
        use_cpu=True,
    )
    .view(-1, perturbed_Y.shape[-1])
    .to(X_test)
)

# Nominal evaluations

Y = mvar_hv.eval_problem(X_test)
infeas = (Y[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
Y = Y[..., : -mvar_hv.num_constraints]
Y[infeas] = mvar_hv.hv.ref_point

In [6]:
plt.scatter(Y[:, 0], Y[:, 1])
plt.scatter(mvar[:, 0], mvar[:, 1])

In [7]:
mvar_per_X = mvar.shape[0] / 100

In [8]:
from botorch.utils.multi_objective.pareto import is_non_dominated

pareto_mask = is_non_dominated(Y)
mvar_p_mask = is_non_dominated(mvar)
Y_p = Y[pareto_mask]
mvar_p = mvar[mvar_p_mask]

In [9]:
plt.scatter(Y_p[:, 0], Y_p[:, 1])
plt.scatter(mvar_p[:, 0], mvar_p[:, 1])

In [10]:
pareto_mask.nonzero()

tensor([[27],
        [54],
        [58],
        [67],
        [74],
        [83],
        [86],
        [95]])

In [11]:
(mvar_p_mask.nonzero() / mvar_per_X).long().unique()

tensor([18, 23, 27, 54, 58, 63, 74, 83, 96])

## There's a lot of overlap here. Let's look at all the points found by MARS-NEI & NParEGO

In [12]:
import os
exp_dir = "../experiments/experiment_v1"

def collect_Xs(problem, label):
    all_Xs = []
    dir_path = os.path.join(exp_dir, problem, label)
    for fp in os.listdir(dir_path):
        fp_path = os.path.join(dir_path, fp)
        try:
            output = torch.load(fp_path)
            all_Xs.append(output["X"])
        except:
            continue
    all_Xs = torch.cat(all_Xs).unique(dim=0)
    print(all_Xs.shape)
    return all_Xs

In [13]:
mars_Xs = collect_Xs("disc_brake", "ref_ch-var-nei")
parego_Xs = collect_Xs("disc_brake", "nparego")

torch.Size([2194, 4])
torch.Size([2197, 4])


In [14]:
parego_Ys = mvar_hv.eval_problem(parego_Xs)
infeas = (parego_Ys[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
parego_Ys = parego_Ys[..., : -mvar_hv.num_constraints]
parego_Ys[infeas] = mvar_hv.hv.ref_point
parego_mask = is_non_dominated(parego_Ys)
parego_p_Ys = parego_Ys[parego_mask]

In [15]:
mars_perturbed_Xs = mvar_hv.perturbation(mars_Xs)
mars_perturbed_Y = mvar_hv.eval_problem(mars_perturbed_Xs)
infeas = (mars_perturbed_Y[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
mars_perturbed_Y = mars_perturbed_Y[..., : -mvar_hv.num_constraints]
mars_perturbed_Y[infeas] = mvar_hv.hv.ref_point

mars_mvar = (
    mvar_hv.mvar(
        mars_perturbed_Y.cpu(),
        use_cpu=True,
    )
    .view(-1, mars_perturbed_Y.shape[-1])
    .to(mars_Xs)
)
mars_mask = is_non_dominated(mars_mvar)
mars_p_mvar = mars_mvar[mars_mask]

In [16]:
plt.scatter(parego_p_Ys[:, 0], parego_p_Ys[:, 1])
plt.title("qParEGO Nominal PF")

Text(0.5, 1.0, 'qParEGO Nominal PF')

In [17]:
plt.scatter(mars_p_mvar[:, 0], mars_p_mvar[:, 1])
plt.title("MARS-NEI MVaR PF")

Text(0.5, 1.0, 'MARS-NEI MVaR PF')

# Let's just look at the joint Xs

In [18]:
Xs = torch.cat([mars_Xs, parego_Xs])

Ys = mvar_hv.eval_problem(Xs)
infeas = (Ys[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
Ys = Ys[..., : -mvar_hv.num_constraints]
Ys[infeas] = mvar_hv.hv.ref_point
y_mask = is_non_dominated(Ys)
p_Ys = Ys[y_mask]


perturbed_Xs = mvar_hv.perturbation(Xs)
perturbed_Ys = mvar_hv.eval_problem(perturbed_Xs)
infeas = (perturbed_Ys[..., -mvar_hv.num_constraints :] < 0).any(dim=-1)
perturbed_Ys = perturbed_Ys[..., : -mvar_hv.num_constraints]
perturbed_Ys[infeas] = mvar_hv.hv.ref_point

mvar = (
    mvar_hv.mvar(
        perturbed_Ys.cpu(),
        use_cpu=True,
    )
    .view(-1, perturbed_Ys.shape[-1])
    .to(Xs)
)
mvar_mask = is_non_dominated(mvar)
p_mvar = mvar[mvar_mask]

Let's find the nominal PF of the points on the MVaR PF.

In [19]:
mvar_per_X = mvar.shape[0] // Xs.shape[0]
mvar_pf_idcs = mvar_mask.nonzero() // mvar_per_X
mvar_pf_Ys = Ys[mvar_pf_idcs].squeeze()

/opt/anaconda3/envs/robust_mobo/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [20]:
target = torch.tensor([-2.55, -2])
(p_mvar > target).all(dim=-1).sum()

tensor(2)

In [21]:
(mvar > target).all(dim=-1).nonzero() // mvar_per_X

tensor([[2081],
        [2081]])

In [22]:
mvar_picked_idx = 2081
selected_mvar_X = Xs[mvar_picked_idx]
selected_mvar_Y = Ys[mvar_picked_idx]

Find points that dominate it in the nominal values.

In [23]:
(Ys > selected_mvar_Y).all(dim=-1).nonzero()

tensor([[ 217],
        [2100],
        [2213],
        [2363],
        [2522],
        [2523],
        [2524],
        [2525],
        [2535],
        [2720],
        [2779],
        [2783],
        [2920],
        [2947],
        [3179],
        [3409],
        [3429],
        [3430],
        [3465],
        [3491],
        [3505],
        [3547],
        [3555],
        [3604],
        [3614],
        [3630],
        [3645],
        [3655],
        [3664],
        [3674],
        [3768],
        [3770],
        [3789],
        [3842],
        [3846],
        [3920],
        [3996],
        [4032],
        [4033],
        [4091],
        [4150],
        [4227],
        [4236],
        [4248],
        [4260],
        [4272],
        [4342],
        [4346],
        [4352],
        [4379]])

Pick one randomly.

In [24]:
picked = 3996
selected_nominal_X = Xs[picked]
selected_nominal_Y = Ys[picked]

Calculate the yields of the two

In [25]:
selected_mvar_perturbed_Ys = perturbed_Ys[mvar_picked_idx * 512: (mvar_picked_idx+1) * 512]
selected_nominal_perturbed_Ys = perturbed_Ys[picked * 512: (picked + 1) * 512]

selected_mvar_yield = (selected_mvar_perturbed_Ys > target).all(dim=-1).sum() / 512.0
selected_nominal_yield = (selected_nominal_perturbed_Ys > target).all(dim=-1).sum() / 512.0

print(f"MVaR yield: {selected_mvar_yield}, nominal yield: {selected_nominal_yield}")

MVaR yield: 0.953125, nominal yield: 0.58203125


Get all the Y's for both picked values, without clamping to the ref pt.

In [26]:
p_Xs = mvar_hv.perturbation(selected_nominal_X.unsqueeze(0))
selected_nominal_raw_Ys = mvar_hv.eval_problem(p_Xs)
selected_nominal_infeas = (selected_nominal_raw_Ys[..., -mvar_hv.num_constraints:] < 0).any(dim=-1)
selected_nominal_raw_Ys = selected_nominal_raw_Ys[..., : -mvar_hv.num_constraints]
selected_nominal_infeas = torch.logical_or(selected_nominal_infeas, (selected_nominal_raw_Ys < target).any(dim=-1))

p_Xs = mvar_hv.perturbation(selected_mvar_X.unsqueeze(0))
selected_mvar_raw_Ys = mvar_hv.eval_problem(p_Xs)
selected_mvar_infeas = (selected_mvar_raw_Ys[..., -mvar_hv.num_constraints:] < 0).any(dim=-1)
selected_mvar_raw_Ys = selected_mvar_raw_Ys[..., : -mvar_hv.num_constraints]
selected_mvar_infeas = torch.logical_or(selected_mvar_infeas, (selected_mvar_raw_Ys < target).any(dim=-1))

## Collect all the plotting here

In [27]:
rc('font', family='serif', style='normal', variant='normal', weight='normal', stretch='normal', size=8)
matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True
matplotlib.style.use('default')
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['axes.titlesize'] = 14
matplotlib.rcParams['axes.labelsize'] = 12
matplotlib.rcParams['legend.fontsize'] = 10
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.pyplot.title(r'ABC123 vs $\mathrm{ABC123}^{123}$')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}\usepackage{bm}"]

default_figsize = (5, 4)

/var/folders/t3/xhtsf2011qndtzgvjdgdnylw0000gn/T/ipykernel_2023/2616554393.py:14: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}\usepackage{bm}"]


In [28]:
from matplotlib.ticker import FormatStrFormatter

fig, ax = plt.subplots(figsize=default_figsize)

obj_vals = "Values"
ss=5
p_Y_color = "deepskyblue"
p_mvar_color = "orange"
mvar_pf_color = "gold"
ax.scatter(p_Ys[:, 0], p_Ys[:, 1], color=p_Y_color,
           label=f"PF over Nominal {obj_vals}", s=ss)
ax.scatter(p_mvar[:, 0], p_mvar[:, 1], color=p_mvar_color,
           label="MVaR", s=ss)
ax.scatter(mvar_pf_Ys[:, 0], mvar_pf_Ys[:, 1], color=mvar_pf_color,
           label=f"Nominal {obj_vals} of MVaR Optimal Designs", s=ss)

ax.legend()
# ax.grid()

s=200
robust_color = "red"
ax.scatter(target[0], target[1], marker="*", s=s, color=robust_color,
           label="Target Specification", edgecolors= "black")

selected_mvar_color = robust_color
ax.scatter(selected_mvar_Y[0], selected_mvar_Y[1], marker="^", s=s,
           label=f"Nominal {obj_vals} of the Robust Design", color=robust_color, edgecolors= "black")

nominal_color = "mediumblue"
ax.scatter(selected_nominal_Y[0], selected_nominal_Y[1], marker="p", s=s,
           label=f"Nominal {obj_vals} of the Non-robust Design", color=nominal_color, edgecolors= "black")

add_shade = True
shade_color = "darkgray"
shade_alpha = 0.2
if add_shade:
    # Shade the area dominating the target.
    ylim = ax.get_ylim()
    x_lim = ax.get_xlim()
    y_min = 1 - (ylim[1] - target[1]) / (ylim[1] - ylim[0])
    ax.axvspan(xmin=float(target[0]), xmax=x_lim[1], ymin=float(y_min), color=shade_color, alpha=shade_alpha,
               label="Area Meeting Target Specification")
    ax.set_xlim(x_lim)
    ax.set_ylim(ylim)

# ax.set_title("Yield Loss from Picking a Non-Robust Solution")
ax.set_title("Selecting Robust and Non-Robust Designs")

ax.set_xlabel(r"Objective 1")
ax.set_ylabel(r"Objective 2")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax.legend()
plt.tight_layout()
plt.savefig(f"plots/yield_1.pdf", dpi=300, bbox_inches='tight')
fig.show()


fig, ax = plt.subplots(figsize=default_figsize)

# ax.grid()
truncated_legend = True
if truncated_legend:
    no_legend_str = str()
    ax.scatter(p_Ys[:, 0], p_Ys[:, 1], label=no_legend_str, s=ss, color=p_Y_color)
    ax.scatter(p_mvar[:, 0], p_mvar[:, 1], label=no_legend_str, s=ss, color=p_mvar_color)
    ax.scatter(mvar_pf_Ys[:, 0], mvar_pf_Ys[:, 1], label=no_legend_str, s=ss, color=mvar_pf_color)
    ax.scatter(target[0], target[1], marker="*", s=s, label=no_legend_str, edgecolors= "black", color=robust_color)
    ax.scatter(selected_mvar_Y[0], selected_mvar_Y[1], marker="^", s=s, label=no_legend_str, color=selected_mvar_color, edgecolors= "black")
    ax.scatter(selected_nominal_Y[0], selected_nominal_Y[1], marker="p", s=s, label=no_legend_str, color=nominal_color, edgecolors= "black")
else:
    raise NotImplementedError

ax.set_title("Yield Loss from Picking a Non-Robust Solution")

# Shade the area dominating the target.
ylim = (-2.1, -1.4)
x_lim = (-2.7, -1.75)
y_min = 1 - (ylim[1] - target[1]) / (ylim[1] - ylim[0])
ax.axvspan(
    xmin=float(target[0]), xmax=x_lim[1], ymin=float(y_min), color=shade_color,
    alpha=shade_alpha, label="Area Meeting Target Specification"
)
ax.set_xlim(x_lim)
ax.set_ylim(ylim)


# infeasible_mvar_color = "orchid"
# infeasible_nominal_color = "lightsalmon"
infeasible_mvar_color = selected_mvar_color
infeasible_nominal_color = nominal_color
feasible_mvar_marker = "+"
infeasible_mvar_marker = "_"
feasible_nominal_marker = feasible_mvar_marker
infeasible_nominal_marker = infeasible_mvar_marker
common_kwargs = {
    "s": 25,
    "alpha": 0.7,
    "linewidths": 1,
}

n_w = 128
mvar_feas = selected_mvar_raw_Ys[:n_w][~selected_mvar_infeas[:n_w]]
mvar_infeas = selected_mvar_raw_Ys[:n_w][selected_mvar_infeas[:n_w]]
nominal_feas = selected_nominal_raw_Ys[:n_w][~selected_nominal_infeas[:n_w]]
nominal_infeas = selected_nominal_raw_Ys[:n_w][selected_nominal_infeas[:n_w]]

# MVaR
dist_str = "Dist."
ax.scatter(
    mvar_feas[:, 0],
    mvar_feas[:, 1],
    marker=feasible_mvar_marker,
    color=selected_mvar_color, label=f"{dist_str} of the Robust Design, Feasible",
    **common_kwargs,
)
ax.scatter(
    mvar_infeas[:, 0],
    mvar_infeas[:, 1],
    marker=infeasible_mvar_marker,
    color=infeasible_mvar_color, label=f"{dist_str} of the Robust Design, Infeasible",
    **common_kwargs,
)
# Nominal
ax.scatter(
    nominal_feas[:, 0],
    nominal_feas[:, 1],
    marker=feasible_nominal_marker,
    color=nominal_color, label=f"{dist_str} of the Non-Robust Design, Feasible",
    **common_kwargs,
)
ax.scatter(
    nominal_infeas[:, 0],
    nominal_infeas[:, 1],
    marker=infeasible_nominal_marker,
    color=infeasible_nominal_color, label=f"{dist_str} of the Non-Robust Design, Infeasible",
    **common_kwargs,
)

ax.set_xlabel(r"Objective 1")
ax.set_ylabel(r"Objective 2")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.legend()
plt.tight_layout()
plt.savefig(f"plots/yield_2.pdf", dpi=300, bbox_inches='tight')
fig.show()


## Single plot version

In [31]:
matplotlib.rcParams['xtick.labelsize'] = 8
matplotlib.rcParams['ytick.labelsize'] = 8
matplotlib.rcParams['axes.titlesize'] = 10
matplotlib.rcParams['axes.labelsize'] = 8
matplotlib.rcParams['legend.fontsize'] = 9


fig, ax = plt.subplots(figsize=default_figsize)

obj_vals = "Values"
ss=5
p_Y_color = "deepskyblue"
p_mvar_color = "orange"
mvar_pf_color = "gold"
ax.scatter(p_Ys[:, 0], p_Ys[:, 1], color=p_Y_color,
           label=f"PF over Nominal {obj_vals}", s=ss)
ax.scatter(p_mvar[:, 0], p_mvar[:, 1], color=p_mvar_color,
           label="MVaR", s=ss)
ax.scatter(mvar_pf_Ys[:, 0], mvar_pf_Ys[:, 1], color=mvar_pf_color,
           label=f"Nominal {obj_vals} of MVaR Optimal Designs", s=ss)

ax.legend()
# ax.grid()

s=80
common_kwargs = {"linewidth": 2, "edgecolors": "black", "s": s}
# robust_color = "red"
robust_color = "tomato"
ax.scatter(target[0], target[1], marker="*", color=robust_color,
           label="Target Specification", **common_kwargs)

selected_mvar_color = robust_color
ax.scatter(selected_mvar_Y[0], selected_mvar_Y[1], marker="^",
           label=f"Nominal {obj_vals} of the Robust Design", color=robust_color, **common_kwargs)

# nominal_color = "mediumblue"
nominal_color = "darkslategrey"
ax.scatter(selected_nominal_Y[0], selected_nominal_Y[1], marker="p",
           label=f"Nominal {obj_vals} of the Non-robust Design", color=nominal_color, **common_kwargs)

# ax.set_title("Yield Loss from Picking a Non-Robust Solution")


# Shade the area dominating the target.
shade_color = "darkgray"
shade_alpha = 0.2
ylim = (-2.1, -1.4)
x_lim = (-2.7, -1.75)
y_min = 1 - (ylim[1] - target[1]) / (ylim[1] - ylim[0])
ax.axvspan(
    xmin=float(target[0]), xmax=x_lim[1], ymin=float(y_min), color=shade_color,
    alpha=shade_alpha, label="Area Meeting Target Specification"
)
ax.set_xlim(x_lim)
ax.set_ylim(ylim)


infeasible_mvar_color = selected_mvar_color
infeasible_nominal_color = nominal_color
feasible_mvar_marker = "o"
infeasible_mvar_marker = "o"
feasible_nominal_marker = feasible_mvar_marker
infeasible_nominal_marker = infeasible_mvar_marker
common_kwargs = {
    "s": 15,
    # "alpha": 0.7,
    "linewidths": 1,
}
feasible_alpha = 0.7
infeasible_alpha = 0.2

n_w = 128
mvar_feas = selected_mvar_raw_Ys[:n_w][~selected_mvar_infeas[:n_w]]
mvar_infeas = selected_mvar_raw_Ys[:n_w][selected_mvar_infeas[:n_w]]
nominal_feas = selected_nominal_raw_Ys[:n_w][~selected_nominal_infeas[:n_w]]
nominal_infeas = selected_nominal_raw_Ys[:n_w][selected_nominal_infeas[:n_w]]

# MVaR
dist_str = "Dist."
gpert = r"\diamond"
# fx_robust = r"$\bm{f}(\bm{x}_{Robust} \diamond \bm{\xi})$"
# fx_nonrobust = r"$\bm{f}(\bm{x}_{Non-Robust} \diamond \bm{\xi})$"
fx_robust = f"{dist_str} of Robust Design"
fx_nonrobust = f"{dist_str} of Non-Robust Design"
ax.scatter(
    mvar_feas[:, 0],
    mvar_feas[:, 1],
    marker=feasible_mvar_marker,
    color=selected_mvar_color,
    label=f"{fx_robust}, Feasible",
    **common_kwargs, alpha=feasible_alpha,
)
ax.scatter(
    mvar_infeas[:, 0],
    mvar_infeas[:, 1],
    marker=infeasible_mvar_marker,
    color=infeasible_mvar_color,
    label=f"{fx_robust}, Infeasible",
    **common_kwargs, alpha=infeasible_alpha,
)
# Nominal
ax.scatter(
    nominal_feas[:, 0],
    nominal_feas[:, 1],
    marker=feasible_nominal_marker,
    color=nominal_color,
    label=f"{fx_nonrobust}, Feasible",
    **common_kwargs, alpha=feasible_alpha,
)
ax.scatter(
    nominal_infeas[:, 0],
    nominal_infeas[:, 1],
    marker=infeasible_nominal_marker,
    color=infeasible_nominal_color,
    label=f"{fx_nonrobust}, Infeasible",
    **common_kwargs, alpha=infeasible_alpha,
)

ax.set_xlabel(r"Objective 1")
ax.set_ylabel(r"Objective 2")
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
h, l = ax.get_legend_handles_labels()
new_h = h[1:7] + h[:1] + h[7:]
new_l = l[1:7] + l[:1] + l[7:]
leg = ax.legend(
    handles=new_h,
    labels=new_l,
    loc="lower center",
    # loc="upper center",
    ncol=2,
    bbox_to_anchor=(0.45,-0.95,0,0),
    # bbox_to_anchor=(0.45,1.8,0,0),
    columnspacing=0.5,
    borderpad=0.1,
    handletextpad=0.5,
    handlelength=1,
    # **{'fontsize': 7.2}
)

ylim = (ylim[0], -1.7)
ax.set_ylim(ylim)
ax.set_aspect(1)


# fig.tight_layout(rect=[0, 0, 1, 1])
fig.tight_layout()
plt.savefig(f"plots/yield_single.pdf", dpi=300, bbox_inches='tight')
fig.show()


## Side-by-side legend version

In [30]:
matplotlib.rcParams['xtick.labelsize'] = 9
matplotlib.rcParams['ytick.labelsize'] = 9
matplotlib.rcParams['axes.titlesize'] = 11
matplotlib.rcParams['axes.labelsize'] = 9
matplotlib.rcParams['legend.fontsize'] = 10

fig, ax = plt.subplots(ncols=1, figsize=(10, 3))

# ax = axes[0]
obj_vals = "Values"
ss=5
p_Y_color = "deepskyblue"
p_mvar_color = "orange"
mvar_pf_color = "gold"
ax.scatter(p_Ys[:, 0], p_Ys[:, 1], color=p_Y_color,
           label=f"PF over Nominal {obj_vals}", s=ss)
ax.scatter(p_mvar[:, 0], p_mvar[:, 1], color=p_mvar_color,
           label="MVaR", s=ss)
ax.scatter(mvar_pf_Ys[:, 0], mvar_pf_Ys[:, 1], color=mvar_pf_color,
           label=f"Nominal {obj_vals} of MVaR Optimal Designs", s=ss)

ax.legend()
# ax.grid()

s=200
common_kwargs = {"linewidth": 3, "edgecolors": "black", "s": s}
# robust_color = "red"
robust_color = "tomato"
ax.scatter(target[0], target[1], marker="*", color=robust_color,
           label="Target Specification", **common_kwargs)

selected_mvar_color = robust_color
ax.scatter(selected_mvar_Y[0], selected_mvar_Y[1], marker="^",
           label=f"Nominal {obj_vals} of the Robust Design", color=robust_color, **common_kwargs)

# nominal_color = "mediumblue"
nominal_color = "darkslategrey"
ax.scatter(selected_nominal_Y[0], selected_nominal_Y[1], marker="p",
           label=f"Nominal {obj_vals} of the Non-robust Design", color=nominal_color, **common_kwargs)

# ax.set_title("Yield Loss from Picking a Non-Robust Solution")


# Shade the area dominating the target.
shade_color = "darkgray"
shade_alpha = 0.2
ylim = (-2.1, -1.4)
x_lim = (-2.7, -1.75)
y_min = 1 - (ylim[1] - target[1]) / (ylim[1] - ylim[0])
ax.axvspan(
    xmin=float(target[0]), xmax=x_lim[1], ymin=float(y_min), color=shade_color,
    alpha=shade_alpha, label="Area Meeting Target Specification"
)
ax.set_xlim(x_lim)
ax.set_ylim(ylim)


infeasible_mvar_color = selected_mvar_color
infeasible_nominal_color = nominal_color
feasible_mvar_marker = "o"
infeasible_mvar_marker = "o"
feasible_nominal_marker = feasible_mvar_marker
infeasible_nominal_marker = infeasible_mvar_marker
common_kwargs = {
    "s": 15,
    # "alpha": 0.7,
    "linewidths": 1,
}
feasible_alpha = 0.7
infeasible_alpha = 0.2

n_w = 128
mvar_feas = selected_mvar_raw_Ys[:n_w][~selected_mvar_infeas[:n_w]]
mvar_infeas = selected_mvar_raw_Ys[:n_w][selected_mvar_infeas[:n_w]]
nominal_feas = selected_nominal_raw_Ys[:n_w][~selected_nominal_infeas[:n_w]]
nominal_infeas = selected_nominal_raw_Ys[:n_w][selected_nominal_infeas[:n_w]]

# MVaR
dist_str = "Dist."
gpert = r"\diamond"
# fx_robust = r"$\bm{f}(\bm{x}_{Robust} \diamond \bm{\xi})$"
# fx_nonrobust = r"$\bm{f}(\bm{x}_{Non-Robust} \diamond \bm{\xi})$"
fx_robust = f"{dist_str} of Robust Design"
fx_nonrobust = f"{dist_str} of Non-Robust Design"
ax.scatter(
    mvar_feas[:, 0],
    mvar_feas[:, 1],
    marker=feasible_mvar_marker,
    color=selected_mvar_color,
    label=f"{fx_robust}, Feasible",
    **common_kwargs, alpha=feasible_alpha,
)
ax.scatter(
    mvar_infeas[:, 0],
    mvar_infeas[:, 1],
    marker=infeasible_mvar_marker,
    color=infeasible_mvar_color,
    label=f"{fx_robust}, Infeasible",
    **common_kwargs, alpha=infeasible_alpha,
)
# Nominal
ax.scatter(
    nominal_feas[:, 0],
    nominal_feas[:, 1],
    marker=feasible_nominal_marker,
    color=nominal_color,
    label=f"{fx_nonrobust}, Feasible",
    **common_kwargs, alpha=feasible_alpha,
)
ax.scatter(
    nominal_infeas[:, 0],
    nominal_infeas[:, 1],
    marker=infeasible_nominal_marker,
    color=infeasible_nominal_color,
    label=f"{fx_nonrobust}, Infeasible",
    **common_kwargs, alpha=infeasible_alpha,
)

ax.set_xlabel(r"Objective 1")
ax.set_ylabel(r"Objective 2")
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
h, l = ax.get_legend_handles_labels()
new_h = h[1:7] + h[:1] + h[7:]
new_l = l[1:7] + l[:1] + l[7:]

ylim = (ylim[0], -1.7)
ax.set_ylim(ylim)
ax.set_aspect(1)

# ax = axes[1]
# fig.delaxes(ax)
leg = ax.legend(
    handles=new_h,
    labels=new_l,
    loc="upper left",
    ncol=1,
    bbox_to_anchor=(1.0,1.0,0,0),
    columnspacing=0.5,
    borderpad=0.1,
    handletextpad=0.5,
    handlelength=1,
    # **{'fontsize': 7.2}
)

# fig.tight_layout(rect=[0, 0, 1, 1])
fig.tight_layout()
plt.savefig(f"plots/yield_sidebyside.pdf", dpi=300, bbox_inches='tight')
fig.show()